
# 🤖 Aula 8: Criando um Chatbot Autônomo que Aprende com seus Erros

Neste notebook, vamos construir um chatbot que registra feedback do usuário e melhora suas respostas ao longo do tempo.

## 📌 O que faremos:
1. **Entender como funciona um chatbot autônomo e seu aprendizado contínuo**.
2. **Fundamentação matemática**: Ajuste adaptativo de respostas via Gradiente Estocástico.
3. **Implementar um chatbot que aprende com feedback negativo e melhora suas respostas**.
4. **Usar embeddings e FAISS para armazenar conhecimento aprimorado**.
5. **Testar um fluxo de aprendizado real**.

---

## 📖 O que é um Chatbot Autônomo?

Um chatbot autônomo é aquele que **se adapta automaticamente ao longo do tempo**. Ele pode ajustar suas respostas de três formas principais:

1. **Ajuste Reativo**: Modifica respostas imediatamente com base no feedback do usuário.
2. **Ajuste Proativo**: Gera novas respostas e testa variações para encontrar a melhor opção.
3. **Aprendizado Contínuo**: Mantém um histórico de interações e melhora de forma progressiva.

### **Matemática por Trás do Ajuste de Respostas**

Podemos modelar esse aprendizado como um **processo de otimização**, onde tentamos minimizar o erro das respostas ao longo do tempo.

Seja $ R(t) $ a resposta do chatbot no tempo $ t $ e $ F(t) $ o feedback recebido, podemos ajustar a resposta usando Gradiente Estocástico:

$
R(t+1) = R(t) + \alpha \cdot (F(t) - R(t))
$

onde:
- $ \alpha $ é a taxa de aprendizado.
- $ F(t) $ pode ser representado como um score numérico atribuído pelo usuário.
- O chatbot ajusta sua resposta para minimizar a diferença entre $ R(t) $ e $ F(t) $.



In [ ]:

# Instale a biblioteca OpenAI se ainda não estiver instalada
!pip install openai json


In [ ]:

import openai
import json

# Defina sua chave de API (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"


In [ ]:

# Criando um chatbot que recebe feedback e registra erros
def salvar_feedback(pergunta, resposta, feedback):
    data = {"pergunta": pergunta, "resposta": resposta, "feedback": feedback}
    with open("feedback.json", "a") as f:
        json.dump(data, f)
        f.write("\n")

def chatbot_aprendizado(pergunta):
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": pergunta}]
    )["choices"][0]["message"]["content"]

    print("Bot:", resposta)
    feedback = input("Isso ajudou? (Sim/Não): ").strip().lower()
    salvar_feedback(pergunta, resposta, feedback)
    return resposta

# Teste do chatbot
print(chatbot_aprendizado("O que é inteligência artificial?"))


In [ ]:

# Instale as bibliotecas necessárias
!pip install openai faiss-cpu numpy json

import openai
import numpy as np
import faiss
import json

# Defina sua chave de API aqui (substitua 'SUA_CHAVE_AQUI' pela chave real)
openai.api_key = "SUA_CHAVE_AQUI"

# Criar banco de memória vetorial com FAISS
dimensao = 1536  # Tamanho dos embeddings da OpenAI
indice_memoria = faiss.IndexFlatL2(dimensao)  # Banco de dados vetorial
memorias = []  # Lista de perguntas e respostas armazenadas
feedbacks = {}  # Armazena feedback dos usuários

# Função para obter embedding de um texto
def obter_embedding(texto):
    resposta = openai.Embedding.create(input=texto, model="text-embedding-ada-002")
    return np.array(resposta["data"][0]["embedding"], dtype=np.float32)

# Função para armazenar memória e feedback
def armazenar_memoria(texto, resposta, feedback):
    vetor = obter_embedding(texto)
    memorias.append((texto, resposta, feedback))
    indice_memoria.add(np.array([vetor]))
    feedbacks[texto] = feedback

# Função para recuperar memória mais próxima
def recuperar_memoria(texto):
    if len(memorias) == 0:
        return None
    
    vetor = obter_embedding(texto)
    D, I = indice_memoria.search(np.array([vetor]), k=1)
    return memorias[I[0][0]] if D[0][0] < 0.5 else None

# Função de aprendizado contínuo
def ajustar_resposta(pergunta):
    memoria = recuperar_memoria(pergunta)
    if memoria and memoria[2] == "não":
        return "📌 Estou ajustando minha resposta para essa pergunta!"
    return None

# Chatbot que aprende com feedback
def chatbot_autonomo(pergunta):
    resposta_melhorada = ajustar_resposta(pergunta)
    if resposta_melhorada:
        return resposta_melhorada
    
    historico = [{"role": "system", "content": "Você é um assistente que melhora suas respostas com feedback do usuário."},
                 {"role": "user", "content": pergunta}]
    
    resposta = openai.ChatCompletion.create(
        model="gpt-4",
        messages=historico
    )["choices"][0]["message"]["content"]

    print(f"Bot: {resposta}")
    feedback = input("Isso ajudou? (Sim/Não) ").strip().lower()
    armazenar_memoria(pergunta, resposta, feedback)

    return resposta

# Teste de aprendizado contínuo
print(chatbot_autonomo("Me explique aprendizado por reforço."))
print(chatbot_autonomo("Como a IA pode ser usada na medicina?"))


In [ ]:

# Chatbot que ajusta suas respostas com base no feedback do usuário
def chatbot_inteligente(pergunta):
    resposta_aprimorada = melhorar_resposta(pergunta)
    if resposta_aprimorada:
        return resposta_aprimorada
    
    return chatbot_aprendizado(pergunta)

# Testando chatbot aprimorado
print(chatbot_inteligente("Explique aprendizado de máquina."))



## 🎯 Desafio para os alunos!
1. Criar um chatbot que registra feedback de usuários e ajusta suas respostas.
2. Implementar um banco de dados simples para armazenar respostas mal avaliadas.
3. Melhorar a lógica de aprendizado contínuo para gerar respostas mais úteis.

---
